In [43]:
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium import webdriver
import os
from collections import Counter
from dotenv import load_dotenv
load_dotenv()

import nltk
import spacy

# essential entity models downloads
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')

import locationtagger


[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package

In [95]:
twitterUsername = os.getenv('TW_USERNAME')
twitterPassword = os.getenv('TW_PASSWORD')
# twitterUsername = os.getenv('TW_USERNAME')
# /article[1]/div[1]/div[1]/div[2]/div[2]

def getTweetData(tweet):
    try:
        tweetTime = tweet.find_element(
            By.XPATH, ".//time").get_attribute('datetime')
    except NoSuchElementException:
        return
    try:
        tweetImage = tweet.find_element(
            By.XPATH, ".//div[1]/div[1]//div[2]/div[2]//img").get_attribute("src")
    except NoSuchElementException:
        print("no image")
        tweetImage = ""
    tweetText = tweet.find_element(
        By.XPATH, ".//div[1]/div[1]/div[2]/div[2]/div[2]").text

    tweetInfo = [tweetText, tweetTime, tweetImage]
    return tweetInfo


# Setting up driver
driver = webdriver.Chrome()
driver.maximize_window()
driver.get('https://www.twitter.com/login')
sleep(3)
# Finding and inputing username
username = driver.find_element(
    By.XPATH, "//body/div[@id='react-root']/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[5]/label[1]/div[1]/div[2]/div[1]/input[1]")
username.send_keys(twitterUsername)
username.send_keys(Keys.RETURN)
sleep(1)
# Finding and inputing Password
# mypassword = getpass()
password = driver.find_element(
    By.XPATH, "//body/div[@id='react-root']/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[3]/div[1]/label[1]/div[1]/div[2]/div[1]/input[1]")
password.send_keys(twitterPassword)
password.send_keys(Keys.RETURN)
sleep(5)

# Selecting and searching required police section
search = driver.find_element(
    By.XPATH, "//body/div[@id='react-root']/div[1]/div[1]/div[2]/main[1]/div[1]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/form[1]/div[1]/div[1]/div[1]/div[1]/label[1]/div[2]/div[1]/input[1]")
search.send_keys('@TPSOperations')
search.send_keys(Keys.RETURN)
sleep(2)

# Click required page
SearchQuery = driver.find_element(
    By.XPATH, "//span[contains(text(),'Toronto Police Operations')]")
SearchQuery.click()


# Storing Tweet data in list
tweetData = []
tweetIds = set()
lastPos = driver.execute_script("return window.pageYOffset;")
scrolling = True
maxDate = False

while (scrolling and not (maxDate)):
    # Find Tweets
    TPOPageTweets = driver.find_elements(
        By.XPATH, '//article[@data-testid="tweet"]')
    sleep(1)
    for tweet in TPOPageTweets[-100:]:
        currentTweetInfo = getTweetData(tweet)
        if currentTweetInfo:
            tweetId = "".join(currentTweetInfo)
            if tweet not in tweetIds and currentTweetInfo[0] != '' and ":" in currentTweetInfo[0] and 'Good night' not in currentTweetInfo[0] and 'Good afternoon' not in currentTweetInfo[0] and 'Good morning' not in currentTweetInfo[0] and 'Good Night' not in currentTweetInfo[0]:
                tweetIds.add(tweet)
                currentTweetInfo.append(tweetId)
                tweetData.append(currentTweetInfo)
                # print(currentTweetInfo[1])
                if "2023-05-01" in currentTweetInfo[1]:
                    maxDate = True
                    break

    scrollAttempt = 0
    while True:
        driver.execute_script('window.scrollBy(0,3000);')
        sleep(2)
        currPos = driver.execute_script("return window.pageYOffset;")
        if maxDate:
            break
        if lastPos == currPos:
            scrollAttempt += 1
            if scrollAttempt >= 3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            lastPos = currPos
            break

no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image
no image


In [ ]:
# Storing Tweet data in list
tweetData = []
tweetIds = set()
lastPos = driver.execute_script("return window.pageYOffset;")
scrolling = True
maxDate = False

while (scrolling and not (maxDate)):
    # Find Tweets
    TPOPageTweets = driver.find_elements(
        By.XPATH, '//article[@data-testid="tweet"]')
    sleep(1)
    for tweet in TPOPageTweets[-100:]:
        currentTweetInfo = getTweetData(tweet)
        if currentTweetInfo:
            tweetId = "".join(currentTweetInfo)
            if tweet not in tweetIds:
                tweetIds.add(tweet)
                tweetData.append([currentTweetInfo, tweetId])
                # print(currentTweetInfo[1])
                if "2023-04-01" in currentTweetInfo[1]:
                    maxDate = True
                    break

    scrollAttempt = 0
    while True:
        driver.execute_script('window.scrollBy(0,3000);')
        sleep(2)
        currPos = driver.execute_script("return window.pageYOffset;")
        if maxDate:
            break
        if lastPos == currPos:
            scrollAttempt += 1
            if scrollAttempt >= 3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            lastPos = currPos
            break

In [96]:
tweetData

[['MISSING:\nAlsara, 27\n- last seen on March 21, in the Neilson Rd & Ellesmere Rd area\n- reported missing to police on May 8\n- described as 5\'2", 120 lbs., slim build, black hair, brown eyes\n- no clothing description available\n#GO1027999\n^al',
  '2023-05-09T01:32:27.000Z',
  'https://pbs.twimg.com/media/FvplmqlWIBIhwvB?format=jpg&name=360x360',
  'MISSING:\nAlsara, 27\n- last seen on March 21, in the Neilson Rd & Ellesmere Rd area\n- reported missing to police on May 8\n- described as 5\'2", 120 lbs., slim build, black hair, brown eyes\n- no clothing description available\n#GO1027999\n^al2023-05-09T01:32:27.000Zhttps://pbs.twimg.com/media/FvplmqlWIBIhwvB?format=jpg&name=360x360'],
 ['HAZARD:\nLawrence Ave W & Grey Rd\n11:03 am\n-reports of hydro wires down across road\n-e/b lanes blocked\n-@TTCnoticies advised\n-no reports of injuries\n-expect delays \n#GO1025548\n^sc',
  '2023-05-08T15:32:22.000Z',
  '',
  'HAZARD:\nLawrence Ave W & Grey Rd\n11:03 am\n-reports of hydro wires do

In [77]:

# List: catagory (Missing...), Status(Update, Located)
computedData = []
# Getting Catagory:
for i in range(len(tweetData)):
    catagory = {}
    # Location = {}
    catagory["Status"] = tweetData[i][0].splitlines()[0].split(":")[0]
    try:
        catagory["Updates"] = tweetData[i][0].splitlines()[0].split(":")[1]
    except IndexError:
        catagory["Updates"] = ""
    # currentLocation could be the currentIdentity
    currentLocation = tweetData[i][0].splitlines()[1].split(", ")
    if any(char.isdigit() for char in currentLocation):
        catagory["Name"] = currentLocation[0]
        if currentLocation[1].isnumeric():
            catagory["Age"] = currentLocation[1]
        else:
            catagory["Age"] = ""
        description = tweetData[i][0].splitlines()
        del description[0:2]
        #Extracting Location from Description
        lists = tweetData[i][0].splitlines()
        del lists[0:2]
        partOfDescription = lists[0] + " " + lists[1]
        catagory["Location"] = locationtagger.find_locations(text = partOfDescription).other
    else:
        try:
            catagory["Location"] = tweetData[i][0].splitlines()[1]
        except IndexError:
            catagory["Location"] = ""
    
        description = tweetData[i][0].splitlines()
        del description[0:1]


    catagory["Description"] = description
    computedData.append(catagory)

(computedData)
        


[{'Status': 'Emergency in Toronto Call 911~Non-Emergency call 416-808-2222~Anonymous Crime Stoppers tips to prevent/solve crime call toll free 1-800-222-8477 or online http',
  'Updates': '//222tips.com. Account is breaking news Toronto Police operations only. All other updates follow ',
  'Location': '@TorontoPolice',
  'Description': ['@TorontoPolice', ' ^sm']},
 {'Status': 'HAZARD',
  'Updates': '',
  'Location': 'Lawrence Ave W & Grey Rd',
  'Description': ['Lawrence Ave W & Grey Rd',
   '11:03 am',
   '-reports of hydro wires down across road',
   '-e/b lanes blocked',
   '-@TTCnoticies advised',
   '-no reports of injuries',
   '-expect delays ',
   '#GO1025548',
   '^sc']},
 {'Status': 'HAZARD',
  'Updates': '(UPDATE)',
  'Location': 'Lawrence Ave W & Grey Rd',
  'Description': ['Lawrence Ave W & Grey Rd',
   '@Toronto_Fire',
   ' o/s',
   '-lanes reduced to avoid hazard',
   '-e/b down to one lane',
   '-w/b down to one lane',
   '-consider alternate routes',
   '#GO1025548',
 

In [83]:
tweetData[3][2]


'https://pbs.twimg.com/profile_images/1224539699798335491/gOEP7UeU_bigger.jpg'